# Problem-Solving Demo: ANLP Assignment 2

We decided to put our money where our mouth is and demonstrate the principles we covered a couple of weeks ago in action.

## Problem 1: Viterbi Algorithm

<div class="alert alert-block alert-info">
First, implement the Viterbi algorithm for finding the optimal state (tag) sequence given the sequence of observations (words). <br><br>
In order to test your implementation, verify that you compute the correct state sequence for some examples from Eisner's ice cream model (see lecture) for which the solutions are known.<br><br>
Demonstrate that your algorithm computes the correct state sequence for ['3','1','3'] as in the lecture.<br><br>
Make sure that your algorithm is correct before proceeding to the other tasks! In order to do this, please also test your module with a longer observation sequence.
</div>

In [1]:
EISNER_STATES = ['C','H']
EISNER_INITIAL_PROBS = {'C': 0.2, 'H': 0.8}
EISNER_TRANSITIONS = {'C': {'C':0.6, 'H': 0.4}, 'H': {'C':0.3, 'H':0.7}}
EISNER_EMISSIONS = {'C': {'1':0.5,'2':0.4,'3':0.1},'H': {'1':0.2, '2':0.4,'3':0.4}}

## Introducing Assertions

Python's `assert` statement is a very handy way to ensure the code you write passes some sanity checks.
The way it works is very simple: it does nothing if the expression that follows it is `True` and it raises an exception if that expression is `False`.

In [2]:
assert True

In [3]:
assert False

AssertionError: 

Assertions are in fact the way our final solution will be tested, so it's good to also use them while working our way up to it.

## Making the Problem Concrete

Throughout this notebook we will be developing our code against a specific example.

It's the same as the test:
Given the input sequence `["3", "1", "3"]` we expect the corresponding sequence of labels to be `["H", "H", "H"]`.

### Break the Problem Down Further

Even having made the problem concrete we can still simplify it even more.
This will make it easier for us to pivot and get into writing some code.

Let's consider namely the "base case" from the slides about the viterbi algorithm (page 43).
The math $V_1 (j) =  b_j(y_1) \cdot a_{0j}$ in our specific case translates to the following english:

*multiply the initial probability of state "H" by the probability of emitting "3" given the state "H"*

If we look up the numbers from the `EISNER_` constants provided, we arrive at the following numbers:

$$0.8 \cdot 0.4$$

Let's make an assertion about them.

In [5]:
assert base_case('H', '3') == (0.8 * 0.4, 'H')

If we run this cell right away we will be told that `base_case` is undefined. Let's fix that.

In [4]:
def base_case(state, observation):
    return (EISNER_INITIAL_PROBS[state] * EISNER_EMISSIONS[state][observation], state)

Now if we run the assertion cell, it should pass silently. Hooray, we got the first step!

Well, almost. We still need to apply our function to all the states. For now we can simply build a dictionary with it.

In [8]:
prob0 = {state: base_case(state, '3') for state in EISNER_STATES}
prob0

{'C': (0.020000000000000004, 'C'), 'H': (0.32000000000000006, 'H')}

With the base step taken care of, it's time to tackle the inductive step from the slides.
Again, in english that formula reads:

*
For every state in the previous time step take its probabilty/score and multiply it with
* the probability of transitioning from this previous state to the state in the current timestep
* the probability of emitting the current observed token given the current state.

Take the maximum of all these values.
*

Just as with the base case, we can plug actual numbers into this and make an assertion about our concrete example using `prob0` which we already computed.

In [14]:
assert inductive_step(prob0, 'H', '1') == (0.32000000000000006 * 0.7 * 0.2, 'H')

Something about sorting tuples by first element...

In [ ]:
max([(1, 2), (2, 3)])

In [39]:
sorted([(1, 2), (2, 3), (2, 1)])

(2, 3)

In [9]:
def inductive_step(previous_paths, current_state, current_obser):
    return max((previous_paths[state][0]
               * EISNER_TRANSITIONS[state][current_state] 
               * EISNER_EMISSIONS[current_state][current_obser], state)
               for state in previous_paths)

In [11]:
prob1 = {state: inductive_step(prob0, state, '1') for state in prob0}
prob1

{'C': (0.04800000000000001, 'H'), 'H': (0.044800000000000006, 'H')}

In [12]:
prob2 = {state: inductive_step(prob1, state, '3') for state in prob1}
prob2

{'C': (0.0028800000000000006, 'C'), 'H': (0.012544000000000003, 'H')}

In [59]:
print(prob0)
print(prob1)
print(prob2)

{'C': (0.020000000000000004, 'C'), 'H': (0.32000000000000006, 'H')}
{'C': (0.04800000000000001, 'H'), 'H': (0.044800000000000006, 'H')}
{'C': (0.0028800000000000006, 'C'), 'H': (0.012544000000000003, 'H')}


We have basically built our trellis already! Putting it all together is relatively easy.

In [62]:
def trellis(observations):
    trel = [{state: base_case(state, observations[0]) for state in EISNER_STATES}]
    for indx, obs in enumerate(observations[1:], 1):
        trel.append({state: inductive_step(trel[indx - 1], state, obs) for state in trel[indx - 1]})
    return trel

In [63]:
trellis(['3', '1', '3'])

[{'C': (0.020000000000000004, 'C'), 'H': (0.32000000000000006, 'H')},
 {'C': (0.04800000000000001, 'H'), 'H': (0.044800000000000006, 'H')},
 {'C': (0.0028800000000000006, 'C'), 'H': (0.012544000000000003, 'H')}]

Now we need to implement the back-tracking through the trellis.
The algorithm in english looks something like this:

*Look at the trellis.

* for the last row, find max
* for the max, take 2nd tuple elem, that's the pointer
* go to prev row, look up pointer
* in this pointer get the next pointer
* rinse and repeat

All the while we keep collecting the pointers into a list.

This time we don't define any assertions of our own and simply stick to the ones for the `eichner_viterbi` ones.

In [79]:
t[1]

{'C': (0.04800000000000001, 'H'), 'H': (0.044800000000000006, 'H')}

In [82]:
t[1][pointer]

(0.044800000000000006, 'H')

Use tuple unpacking to get the next pointer

In [78]:
(prob, state), pointer = max((val, key) for key, val in t[-1].items())

In [76]:
 max((val, key) for key, val in t[-1].items())

((0.012544000000000003, 'H'), 'H')

In [94]:
def eisner_viterbi(observations):
    t = trellis(observations)
    (prob, state), pointer = max((val, key) for key, val in t[-1].items())
    states = [state]
    for step in reversed(t[:-1]):
        states.append(pointer)
        pointer = step[pointer][1]
    return list(reversed(states))

In [95]:
eisner_viterbi(['3', '1', '3'])

['H', 'H', 'H']

In [96]:
assert eisner_viterbi(["3", "1", "3"]) == ["H", "H", "H"]

# from slack
assert eisner_viterbi(["2", "1", "3", "2", "1", "1", "2"]) == ["H", "H", "H", "H", "C", "C", "C"]

# from slack
assert (eisner_viterbi(["3", "1", "3", "1", "1", "3", "1", "1", "1", "3", "3", "3", "2", "2", "2", "1", "1"]) ==
                       ["H", "H", "H", "C", "C", "H", "C", "C", "C", "H", "H", "H", "H", "H", "H", "C", "C"] )